# 0. before handling database having some installation

In [2]:
pip install pymongo

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


# 객체화

## class : Processing

In [62]:
import pymongo
import pandas as pd
import numpy as np
import copy
from datetime import datetime

class Preprocessing:
    ##########이상치 처리##########
    def pivot_PET(self, dataframe):
        whe = []
        for i in dataframe.columns:
            if '명' in i:
                whe.append(1)
            else:
                whe.append(0)
        whe = np.array(whe)

        dataframe_names = dataframe.columns[np.where(whe == 1)]
        dataframe_amount = dataframe.columns[np.where(whe == 0)]

        dataframe_names_df = dataframe[dataframe_names]
        dataframe_amount_df = dataframe[dataframe_amount]

        dataframe_names_unique = []
        for i in dataframe_names:
            dataframe_names_unique.extend(dataframe[i].unique())
        
        col_names = pd.DataFrame(dataframe_names_unique)[0].unique()
        col_names = col_names[~pd.isnull(col_names)]
        dataframe_reshape = pd.DataFrame(index=dataframe.index, columns=col_names)
        
        for i in range(len(dataframe_reshape)):
            for j in range(len(dataframe_names_df.columns)):
                if pd.isna(dataframe_names_df.loc[i][j]) == False:
                    dataframe_reshape.loc[i][np.where(dataframe_reshape.columns == dataframe_names_df.loc[i][j])[0][0]] = \
                    dataframe_amount_df.loc[i][j]  # [0][0]는고정
                else:
                    continue
        return (dataframe_reshape)
    ###############################
        
    # 행 중복처리
    def distinction(self):
        self.df['keys'] = 0
        list_temp_one = self.df['시료번호']
        list_temp_two = self.df['접수번호No.']
        list_temp_three = []
        format = '%Y-%m-%d'
        for i in range(len(list_temp_one)):
            str_datetime = datetime.strftime(list_temp_two[i], format)
        #     print(str(str_datetime)+'_'+str(list_temp_one[i]))
            list_temp_three.append(str(str_datetime)+'_'+str(list_temp_one[i]))
        # keys 값을 기준으로 중복 제거
        self.df['keys'] = list_temp_three
        unduplicated_data = self.df.drop_duplicates(subset='keys')
        self.extraxt_df = unduplicated_data[['내광성시험분석', 'Pilot 전처리 원단 투입량', 'Pilot 전처리 조제 #1 제품명', 'Pilot 전처리 조제 #1제조사명', 'Pilot 전처리 조제 #1 종류', 'Pilot 전처리 조제 #1투입량',
                                          'Pilot 전처리 조제 #2 제품명', 'Pilot 전처리 조제 #2제조사명', 'Pilot 전처리 조제 #2 종류', 'Pilot 전처리 조제 #2투입량',
                                          'Pilot 전처리 조제 #3 제품명', 'Pilot 전처리 조제 #3제조사명', 'Pilot 전처리 조제 #3 종류', 'Pilot 전처리 조제 #3투입량',
                                          'Lab 배합 시료 무게', 'Lab 배합 염료 #1 명', 'Lab 배합 염료 #1 제조사명', 'Lab 배합 염료 #1 종류', 'Lab 배합 염료 #1 타입', 'Lab 배합 염료 #1 투입량',
                                          'Lab 배합 염료 #2 명', 'Lab 배합 염료 #2 제조사명', 'Lab 배합 염료 #2 종류', 'Lab 배합 염료 #2 타입', 'Lab 배합 염료 #2 투입량',
                                          'Lab 배합 염료 #3 명', 'Lab 배합 염료 #3 제조사명', 'Lab 배합 염료 #3 종류', 'Lab 배합 염료 #3 타입', 'Lab 배합 염료 #3 투입량',
                                          'Lab 배합 염료 #4 명', 'Lab 배합 염료 #4 제조사명', 'Lab 배합 염료 #4 종류', 'Lab 배합 염료 #4 타입', #'Lab 배합 염료 #4 투입량',
                                          'Lab 배합 액비', 'Lab 배합 용수량', 'Lab 배합 조제 #1 명', 'Lab 배합 조제 #1 제조사명', 'Lab 배합 조제 #1 종류', 'Lab 배합 조제 #1 투입량',
                                          'Lab 배합 조제 #2 명', 'Lab 배합 조제 #2 제조사명', 'Lab 배합 조제 #2 종류', 'Lab 배합 조제 #2 투입량',
                                          'Lab 배합 조제 #3 명', 'Lab 배합 조제 #3 제조사명', 'Lab 배합 조제 #3 종류', 'Lab 배합 조제 #3 투입량',
                                          'Lab 배합 조제 #4 명', 'Lab 배합 조제 #4 제조사명', 'Lab 배합 조제 #4 종류', 'Lab 배합 조제 #4 투입량',
                                          'Lab 후처리 조제 #1 명', 'Lab 후처리 조제 #1 제조사명', 'Lab 후처리 조제 #1 종류', 'Lab 후처리 조제 #1 투입량',
                                          'Lab 후처리 조제 #2 명', 'Lab 후처리 조제 #2 제조사명', 'Lab 후처리 조제 #2 종류', 'Lab 후처리 조제 #2 투입량',
                                          'Lab 후처리 환원세정 액비', 'Lab 후처리 환원세정 용수량', '내광성등급'
                                        ]]
        self.extraxt_df = self.extraxt_df.reset_index()

    # 배합 염료
    def compound_dye(self):
        dye_data = self.extraxt_df[['Lab 배합 염료 #1 명','Lab 배합 염료 #1 투입량',
            'Lab 배합 염료 #2 명','Lab 배합 염료 #2 투입량',
            'Lab 배합 염료 #3 명','Lab 배합 염료 #3 투입량',
            'Lab 배합 염료 #4 명']]
            #'Lab 배합 염료 #4 명','Lab 배합 염료 #4 투입량',]]

        self.dye_data_df = self.pivot_PET(dye_data)
        self.dye_data_df = self.dye_data_df.fillna(0)
        
        DBKKL = []
        '''
        for i in range(len(dye_data_df)):
            temp_data = dye_data_df['Dorosperse Black KKL'][i] + dye_data_df['Dorosperse B/K KKL'][i]
            #temp_data = dye_data_df.loc[i, ['Dorosperse Black KKL']] +  dye_data_df.loc[i, ['Dorosperse B/K KKL']]
            #print(temp_data)
            DBKKL.append(temp_data)

        dye_data_df = dye_data_df.drop('Dorosperse B/K KKL', axis=1)
        dye_data_df['Dorosperse Black KKL'] = DBKKL
        '''
    
    # 배합 조제
    def compound_prescription(self):
        prepare_data = self.extraxt_df[['Lab 배합 조제 #1 명', 'Lab 배합 조제 #1 투입량',
                     'Lab 배합 조제 #2 명', 'Lab 배합 조제 #2 투입량',
                     'Lab 배합 조제 #3 명', 'Lab 배합 조제 #3 투입량',
                     'Lab 배합 조제 #4 명', 'Lab 배합 조제 #4 투입량']]
        self.prepare_data_df = self.pivot_PET(prepare_data)
        self.prepare_data_df = self.prepare_data_df.fillna(0)
        
    # 후처리 조제 NaOH - 1
    def post_proecessing(self):
        post_prepare_data = self.extraxt_df[['Lab 후처리 조제 #1 명', 
                                             'Lab 후처리 조제 #1 투입량', 
                                             'Lab 후처리 조제 #2 명', 
                                             'Lab 후처리 조제 #2 투입량']]
        # Lab 후처리
        self.post_prepare_data_df = self.pivot_PET(post_prepare_data)
        self.post_prepare_data_df = self.post_prepare_data_df.fillna(0)
        self.post_prepare_data_df.rename(columns = {'NaOH':'Lab후처리_NaOH'}, inplace = True)
        
    # 전처리 조제 NaOH - 2
    def pre_processing(self):
        pre_prepare_data = self.extraxt_df[['Pilot 전처리 조제 #1 제품명',
                                            'Pilot 전처리 조제 #1투입량',
                                            'Pilot 전처리 조제 #2 제품명',
                                            'Pilot 전처리 조제 #2투입량',
                                            'Pilot 전처리 조제 #3 제품명',
                                            'Pilot 전처리 조제 #3투입량']]
        # Pilot전처리
        self.pre_prepare_data_df = self.pivot_PET(pre_prepare_data)
        self.pre_prepare_data_df = self.pre_prepare_data_df.fillna(0)
        self.pre_prepare_data_df.rename(columns = {'NaOH':'Pilot전처리_NaOH'}, inplace = True)
        
    # 합치기
    def combine(self):
        new_extract_data = self.extraxt_df[['Pilot 전처리 원단 투입량', 
                                            'Lab 배합 시료 무게', 
                                            'Lab 배합 용수량', 
                                            'Lab 후처리 환원세정 용수량', 
                                            '내광성등급']]
        self.new_df = pd.concat([self.prepare_data_df, 
                                 self.pre_prepare_data_df, 
                                 self.dye_data_df, 
                                 self.post_prepare_data_df, 
                                 new_extract_data], axis=1)

## class : Raw 

In [63]:
# raw data를 관리할 수 있는 클래스 생성
class Raw(Preprocessing, RenameRaw): # raw_Data가 존재하는 경로 지정
    def __init__(self):
        conn = pymongo.MongoClient()
        self.test_db = conn.test # db는 고정 (서버에 올릴때 수정)
        self.col_raw = self.test_db.raw_data 
        self.col_prep = self.test_db.processed_data
        # test를 위해 디비를 초기상태로 만들기 위함
        self.col_raw.delete_many({})  
        self.col_prep.delete_many({})
    
    def toJSON(self):
        self.Origin_Jdata = self.df.to_dict(orient='records')
        self.Preprocessed_Jdata = self.new_df.to_dict(orient='records')
    
    def DataTuning(self, path, name):
        # Raw data
        self.df = pd.read_excel(
            path + name,
            header=[0, 1, 2, 3])
        self.classifier = list(map(lambda x : ''.join(x[0]), self.df.columns))
        self.column_list = list(map(lambda x : ''.join(x[1]), self.df.columns))
        self.rename()
#         self.df.columns = self.column_list
        self.df.columns = self.new_col_RAW
            
        # data 처리, 전처리
        self.distinction() # 중복처리
        self.compound_dye()
        self.compound_prescription()
        self.post_proecessing()
        self.pre_processing()
        self.combine()
        
        self.toJSON()
        
        
    def insertDB(self):
        print('Origin before insert : ' + str(self.col_raw.count_documents({})))
        print('Preprocessed before insert : ' + str(self.col_prep.count_documents({})))
        
        self.col_raw.insert_many(self.Origin_Jdata)
        self.col_prep.insert_many(self.Preprocessed_Jdata)
        
        print('Origin after insert : ' + str(self.col_raw.count_documents({})))
        print('Preprocessed after insert : ' + str(self.col_prep.count_documents({})))
    
    def readRAW(self, col=-1, lim=-1): # 조건으로 탐색은 따로 만들예정
        if lim == -1:
            if col == -1:
                allData = self.col_raw.find({}, {'_id' : 0})
            else:
                allData = self.col_raw.find({}, {'_id' : 0, col:1})
        else:
            if col == -1:
                allData = self.col_raw.find({}, {'_id' : 0}).limit(lim)
            else:
                allData = self.col_raw.find({}, {'_id' : 0, col:1}).limit(lim)
        for d in allData:
            print(d)
            
    def readPrep(self, col=-1, lim=-1): # 조건으로 탐색은 따로 만들예정
        if lim == -1:
            if col == -1:
                allData = self.col_prep.find({}, {'_id' : 0})
            else:
                allData = self.col_prep.find({}, {'_id' : 0, col:1})
        else:
            if col == -1:
                allData = self.col_prep.find({}, {'_id' : 0}).limit(lim)
            else:
                allData = self.col_prep.find({}, {'_id' : 0, col:1}).limit(lim)
        for d in allData:
            print(d)
    
#     def deleteAll(self):
# #         self.col_usage.delete_many({})
#         self.col_raw.delete_many({})
    
#     def deleteSeleted(self, cond):
# #         self.col_usage.delete_one(cond)
#         self.col_raw.delete_one(cond)
    
#     def update(self, cond, updateTo):
#         self.col_raw.update_many(cond, {'$set' : updateTo})
# #         self.col_usage.update_many(cond, {'$set' : updateTo})

## class : RenameRAW

In [ ]:
class RenameRaw:
    def rename(self):
        df_cols = list(self.column_list)
        self.new_col_RAW = []
        for i, ds in enumerate(df_cols):
            if df_cols.count(ds) > 1:
                self.new_col_RAW.append(self.classifier[i] + '_' + ds)
            else:
                self.new_col_RAW.append(ds)

# trial for testing DB

In [64]:
# trial
# -----------------------DB 연결------------------------------
p = 'C:/Users/eselab/Desktop/권순범/다이텍 과제 관련 자료/'
# p = 'C:/Users/sunbum/Desktop/연구실/내광성 예측 모델-DB/다이텍 과제 관련 자료/'
name = '기초DB_통합자료(다이텍,부산대)_염색분야v230101_경북대발송분.xlsx'

# 데이터베이스 연동을 위한 설정 맟 객체 생성
data = Raw()

# df로 생성 및 확인 (이후 JSON으로 변환)
data.DataTuning(p, name)

# -----------------------DB 삽입 후 관리------------------------------

# data를 db에 삽입 (삽입 확인)
data.insertDB()

# 특정 feature 조회,
# para1 : 조회할 feature
# para2 : 조회할 value 개수
n = 5
# data.readRAW('시료번호', n)
# data.readPrep('NaOH', n)

# -----------------------추가기능------------------------------



# # 특정 data를 삭제하기 위해 조건(cond1)를 dict로 만들어야 한다
# # 예시) {'특정 column 이름' : 'value 조건'}
# cond1 = {'Lab 배합 염료 #4 투입량' : -1}
# origin_data.deleteSeleted(cond1)

# # 특정 data를 수정하기 위한 조건(cond2)과 수정하려는 내용(updateTo)를 dict로 만들어야 한다
# # 조건에 해당되는 data는 전부 수정된다
# feature = 'Lab 배합 염료 #4 투입량' 
# cond2 = {feature : np.nan}
# updateTo = {feature : -1}
# origin_data.update(cond2, updateTo)
# origin_data.read(feature)
# # 원상태로 
# cond2 = {feature : -1}
# updateTo = {feature : np.nan}
# origin_data.update(cond2, updateTo)
# # origin_data.read(feature, 5)


# NaOH가 있는 두개 df
# data.post_prepare_data_df
# data.pre_prepare_data_df
# list(data.new_df.columns)

print(len(data.col_prep.find_one({},{'_id' : 0})))
print(len(data.col_raw.find_one({}, {'_id' : 0})))

C:\Users\sunbum\AppData\Local\Temp\ipykernel_20992\4122680617.py:13: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  self.Origin_Jdata = self.df.to_dict(orient='records')


# 추가 테스트 및 자료

In [67]:
# NaOH가 있는 두개 df
# data.post_prepare_data_df
# data.pre_prepare_data_df
# list(data.new_df.columns)
len(data.col_prep.find_one({},{'_id' : 0}))

36

In [66]:
# data를 db에 삽입 (삽입 확인)
data.insertDB()

# 특정 feature 조회,
# para1 : 조회할 feature
# para2 : 조회할 value 개수
n = 5
# data.readRAW('시료번호', n)
# data.readPrep('NaOH', n)

# -----------------------추가기능------------------------------



# # 특정 data를 삭제하기 위해 조건(cond1)를 dict로 만들어야 한다
# # 예시) {'특정 column 이름' : 'value 조건'}
# cond1 = {'Lab 배합 염료 #4 투입량' : -1}
# origin_data.deleteSeleted(cond1)

# # 특정 data를 수정하기 위한 조건(cond2)과 수정하려는 내용(updateTo)를 dict로 만들어야 한다
# # 조건에 해당되는 data는 전부 수정된다
# feature = 'Lab 배합 염료 #4 투입량' 
# cond2 = {feature : np.nan}
# updateTo = {feature : -1}
# origin_data.update(cond2, updateTo)
# origin_data.read(feature)
# # 원상태로 
# cond2 = {feature : -1}
# updateTo = {feature : np.nan}
# origin_data.update(cond2, updateTo)
# # origin_data.read(feature, 5)

Origin before insert : 0
Preprocessed before insert : 0
Origin after insert : 6026
Preprocessed after insert : 3215


# 추가 자료
- projection (권순범)
- preprocessing (서강복 박사과정)

In [9]:
import pandas as pd
import numpy as np
from datetime import datetime

# 권순범
import os

def pivot_PET(dataframe):
    whe = []
    for i in dataframe.columns:
        if '명' in i:
            whe.append(1)
        else:
            whe.append(0)
    whe = np.array(whe)

    dataframe_names = dataframe.columns[np.where(whe == 1)]
    dataframe_amount = dataframe.columns[np.where(whe == 0)]

    dataframe_names_df = dataframe[dataframe_names]
    dataframe_amount_df = dataframe[dataframe_amount]

    dataframe_names_unique = []
    for i in dataframe_names:
        dataframe_names_unique.extend(dataframe[i].unique())

    col_names = pd.DataFrame(dataframe_names_unique)[0].unique()
    col_names = col_names[~pd.isnull(col_names)]
    dataframe_reshape = pd.DataFrame(index=dataframe.index, columns=col_names)

    for i in range(len(dataframe_reshape)):
        for j in range(len(dataframe_names_df.columns)):
            if pd.isna(dataframe_names_df.loc[i][j]) == False:
                dataframe_reshape.loc[i][np.where(dataframe_reshape.columns == dataframe_names_df.loc[i][j])[0][0]] = \
                dataframe_amount_df.loc[i][j]  # [0][0]는고정
            else:
                continue
    return (dataframe_reshape)




#data = pd.read_excel('E:/과제/다이텍/추가데이터_20230102/기초DB_통합자료(다이텍,부산대)_염색분야v230101_경북대발송분_data_python.xlsx')
# data = pd.read_excel('C:/Users/eselab/Desktop/권순범/다이텍 과제 관련 자료/기초DB_통합자료(다이텍,부산대)_염색분야v230101_경북대발송분.xlsx')
p = 'C:/Users/sunbum/Desktop/연구실/내광성 예측 모델-DB/다이텍 과제 관련 자료/'
name = '기초DB_통합자료(다이텍,부산대)_염색분야v230101_경북대발송분.xlsx'
data = pd.read_excel(p + name)

data=data[data.columns[1:]]
#data=data.drop([1,],axis=0)
#data=data.drop([1,],axis=0)
data=data.drop([1],axis=0)
data=data.drop([2],axis=0)
data = data.rename(columns=data.iloc[0])
data=data.drop([0],axis=0)
data = data.reset_index()
#print(data)
# columns명 지정 및 쓸모없는 열 삭제
# 중복 제거용 key 값 생성 (시료번호 + 접수번호No.)

data['keys'] = 0
list_temp_one = data['시료번호']
list_temp_two = data['접수번호No.']
list_temp_three = []
format = '%Y-%m-%d'

for i in range(len(list_temp_one)):
    str_datetime = datetime.strftime(list_temp_two[i], format)
    #print(str(str_datetime)+'_'+str(list_temp_one[i]))
    list_temp_three.append(str(str_datetime)+'_'+str(list_temp_one[i]))

# keys 값을 기준으로 중복 제거
data['keys'] = list_temp_three
#print(data['keys'])
unduplicated_data = data.drop_duplicates(subset='keys')
#print(unduplicated_data)

# 2023_01_09 현재 사용 데이터 추출 67개
# 여기서 '번호'와 '시료번호', '접수번호No.' 제외하고 keys값으로 대체
#번호/시료번호/내광성시험분석/접수번호No./Pilot 전처리 원단 투입량/Pilot 전처리 조제 #1 제품명/Pilot 전처리 조제 #1제조사명/Pilot 전처리 조제 #1 종류/Pilot 전처리 조제 #1투입량
#Pilot 전처리 조제 #2 제품명/Pilot 전처리 조제 #2제조사명/Pilot 전처리 조제 #2 종류/Pilot 전처리 조제 #2투입량
#Pilot 전처리 조제 #3 제품명/Pilot 전처리 조제 #3제조사명/Pilot 전처리 조제 #3 종류/Pilot 전처리 조제 #3투입량
#Lab 배합 시료 무게/Lab 배합 염료 #1 명/Lab 배합 염료 #1 제조사명/Lab 배합 염료 #1 종류/Lab 배합 염료 #1 타입/Lab 배합 염료 #1 투입량
#Lab 배합 염료 #2 명/Lab 배합 염료 #2 제조사명/Lab 배합 염료 #2 종류/Lab 배합 염료 #2 타입/Lab 배합 염료 #2 투입량

#Lab 배합 염료 #3 명/Lab 배합 염료 #3 제조사명/Lab 배합 염료 #3 종류/Lab 배합 염료 #3 타입/Lab 배합 염료 #3 투입량
#Lab 배합 염료 #4 명/Lab 배합 염료 #4 제조사명/Lab 배합 염료 #4 종류/Lab 배합 염료 #4 타입/Lab 배합 염료 #4 투입량
#Lab 배합 액비/Lab 배합 용수량/Lab 배합 조제 #1 명/Lab 배합 조제 #1 제조사명/Lab 배합 조제 #1 종류/Lab 배합 조제 #1 투입량
#Lab 배합 조제 #2 명/Lab 배합 조제 #2 제조사명/Lab 배합 조제 #2 종류/Lab 배합 조제 #2 투입량
#Lab 배합 조제 #3 명/Lab 배합 조제 #3 제조사명/Lab 배합 조제 #3 종류/Lab 배합 조제 #3 투입량

#Lab 배합 조제 #4 명/Lab 배합 조제 #4 제조사명/Lab 배합 조제 #4 종류/Lab 배합 조제 #4 투입량
#Lab 후처리 조제 #1 명/Lab 후처리 조제 #1 제조사명/Lab 후처리 조제 #1 종류/Lab 후처리 조제 #1 투입량
#Lab 후처리 조제 #2 명/Lab 후처리 조제 #2 제조사명/Lab 후처리 조제 #2 종류/Lab 후처리 조제 #2 투입량
#Lab 후처리 환원세정 액비/Lab 후처리 환원세정 용수량/내광성등급


extraxt_data = unduplicated_data[['내광성시험분석', 'Pilot 전처리 원단 투입량', 'Pilot 전처리 조제 #1 제품명', 'Pilot 전처리 조제 #1제조사명', 'Pilot 전처리 조제 #1 종류', 'Pilot 전처리 조제 #1투입량',
                                  'Pilot 전처리 조제 #2 제품명', 'Pilot 전처리 조제 #2제조사명', 'Pilot 전처리 조제 #2 종류', 'Pilot 전처리 조제 #2투입량',
                                  'Pilot 전처리 조제 #3 제품명', 'Pilot 전처리 조제 #3제조사명', 'Pilot 전처리 조제 #3 종류', 'Pilot 전처리 조제 #3투입량',
                                  'Lab 배합 시료 무게', 'Lab 배합 염료 #1 명', 'Lab 배합 염료 #1 제조사명', 'Lab 배합 염료 #1 종류', 'Lab 배합 염료 #1 타입', 'Lab 배합 염료 #1 투입량',
                                  'Lab 배합 염료 #2 명', 'Lab 배합 염료 #2 제조사명', 'Lab 배합 염료 #2 종류', 'Lab 배합 염료 #2 타입', 'Lab 배합 염료 #2 투입량',
                                  'Lab 배합 염료 #3 명', 'Lab 배합 염료 #3 제조사명', 'Lab 배합 염료 #3 종류', 'Lab 배합 염료 #3 타입', 'Lab 배합 염료 #3 투입량',
                                  'Lab 배합 염료 #4 명', 'Lab 배합 염료 #4 제조사명', 'Lab 배합 염료 #4 종류', 'Lab 배합 염료 #4 타입', #'Lab 배합 염료 #4 투입량',
                                  'Lab 배합 액비', 'Lab 배합 용수량', 'Lab 배합 조제 #1 명', 'Lab 배합 조제 #1 제조사명', 'Lab 배합 조제 #1 종류', 'Lab 배합 조제 #1 투입량',
                                  'Lab 배합 조제 #2 명', 'Lab 배합 조제 #2 제조사명', 'Lab 배합 조제 #2 종류', 'Lab 배합 조제 #2 투입량',
                                  'Lab 배합 조제 #3 명', 'Lab 배합 조제 #3 제조사명', 'Lab 배합 조제 #3 종류', 'Lab 배합 조제 #3 투입량',
                                  'Lab 배합 조제 #4 명', 'Lab 배합 조제 #4 제조사명', 'Lab 배합 조제 #4 종류', 'Lab 배합 조제 #4 투입량',
                                  'Lab 후처리 조제 #1 명', 'Lab 후처리 조제 #1 제조사명', 'Lab 후처리 조제 #1 종류', 'Lab 후처리 조제 #1 투입량',
                                  'Lab 후처리 조제 #2 명', 'Lab 후처리 조제 #2 제조사명', 'Lab 후처리 조제 #2 종류', 'Lab 후처리 조제 #2 투입량',
                                  'Lab 후처리 환원세정 액비', 'Lab 후처리 환원세정 용수량', '내광성등급'
                                ]]
extraxt_data = extraxt_data.reset_index()
#print(extraxt_data)

extraxt_data = extraxt_data.drop('index', axis=1)
#print(extraxt_data)
# extraxt_data.to_csv('E:/과제/다이텍/추가데이터_20230119/unduplicated_data.csv', encoding='euc-kr')
# extraxt_data = pd.read_csv('E:/과제/다이텍/추가데이터_20230119/unduplicated_data.csv',encoding='cp949')
dye_data = extraxt_data[['Lab 배합 염료 #1 명','Lab 배합 염료 #1 투입량',
            'Lab 배합 염료 #2 명','Lab 배합 염료 #2 투입량',
            'Lab 배합 염료 #3 명','Lab 배합 염료 #3 투입량',
            'Lab 배합 염료 #4 명']]
            #'Lab 배합 염료 #4 명','Lab 배합 염료 #4 투입량',]]

dye_data_df = pivot_PET(dye_data)
dye_data_df=dye_data_df.fillna(0)
#print(dye_data_df.columns)
#print(dye_data_df)
DBKKL = []
'''
for i in range(len(dye_data_df)):
    temp_data = dye_data_df['Dorosperse Black KKL'][i] + dye_data_df['Dorosperse B/K KKL'][i]
    #temp_data = dye_data_df.loc[i, ['Dorosperse Black KKL']] +  dye_data_df.loc[i, ['Dorosperse B/K KKL']]
    #print(temp_data)
    DBKKL.append(temp_data)

dye_data_df = dye_data_df.drop('Dorosperse B/K KKL', axis=1)
dye_data_df['Dorosperse Black KKL'] = DBKKL
'''
prepare_data = extraxt_data[['Lab 배합 조제 #1 명', 'Lab 배합 조제 #1 투입량',
             'Lab 배합 조제 #2 명', 'Lab 배합 조제 #2 투입량',
             'Lab 배합 조제 #3 명', 'Lab 배합 조제 #3 투입량',
             'Lab 배합 조제 #4 명', 'Lab 배합 조제 #4 투입량']]
prepare_data_df = pivot_PET(prepare_data)
prepare_data_df = prepare_data_df.fillna(0)
#print(prepare_data_df.columns)
#print(prepare_data_df)

post_prepare_data = extraxt_data[['Lab 후처리 조제 #1 명', 'Lab 후처리 조제 #1 투입량', 'Lab 후처리 조제 #2 명', 'Lab 후처리 조제 #2 투입량']]
post_prepare_data_df = pivot_PET(post_prepare_data)
post_prepare_data_df = post_prepare_data_df.fillna(0)
#print(post_prepare_data_df.columns)
#print(post_prepare_data_df)

pre_prepare_data = extraxt_data[['Pilot 전처리 조제 #1 제품명','Pilot 전처리 조제 #1투입량',
            'Pilot 전처리 조제 #2 제품명','Pilot 전처리 조제 #2투입량',
           'Pilot 전처리 조제 #3 제품명','Pilot 전처리 조제 #3투입량']]

pre_prepare_data_df = pivot_PET(pre_prepare_data)
pre_prepare_data_df = pre_prepare_data_df.fillna(0)

new_extract_data = extraxt_data[['Pilot 전처리 원단 투입량', 'Lab 배합 시료 무게', 'Lab 배합 용수량', 'Lab 후처리 환원세정 용수량', '내광성등급']]
new_data = pd.concat([prepare_data_df, pre_prepare_data_df, dye_data_df, post_prepare_data_df,  new_extract_data], axis=1)
#print(new_data.columns)
#new_data.to_csv('E:/과제/다이텍/추가데이터_20230102/preprocsssintg_data.csv', encoding='euc-kr')

now = datetime.now()
res_folder = "전처리_" + now.strftime('%Y_%m_%d')
os.mkdir(p + res_folder)


new_data.to_csv(p+res_folder+'/preprocsssintg_data.csv', encoding='euc-kr')
# 이까지가 전처리 조제, 염료, 조제, 후처리 조제, 추가적인 데이터까지 데이터 셋 생성한 것임

#df_corr = new_data.corr(method='pearson')
#df_corr.to_csv('E:/과제/다이텍/추가데이터_20230119/corr.csv')

KeyError: "['Lab 배합 염료 #4 투입량'] not in index"

In [28]:
unduplicated_data

,index,번호,시료번호,내광성시험분석,접수번호No.,업체명,염색법,기계분류ID,기계분류,원료ID,...,L*,a*,b*,L*,a*,b*,내광성등급,색차,비고,keys
0,3,1,1,NaN,2021-03-18 00:00:00,다이텍연구원,1,P-DY09,고압 래피드 염색기,RAW-001,...,79.15,7.49,63.03,82.19,12.12,62.25,3.5,5.673333,NaN,2021-03-18_1
1,4,2,2,NaN,2021-03-18 00:00:00,다이텍연구원,1,P-DY09,고압 래피드 염색기,RAW-001,...,75.83,14.91,74.03,78.3,18.93,72.91,3.5,4.85,NaN,2021-03-18_2
2,5,3,3,NaN,2021-03-18 00:00:00,다이텍연구원,1,P-DY09,고압 래피드 염색기,RAW-001,...,73.7,18.95,77.57,76.54,22.88,76.71,3.5,4.93,NaN,2021-03-18_3
3,6,4,4,NaN,2021-03-18 00:00:00,다이텍연구원,1,P-DY09,고압 래피드 염색기,RAW-001,...,72.58,21.31,78.93,74.55,26.46,79.48,3.5,5.541,NaN,2021-03-18_4
4,7,5,5,NaN,2021-03-18 00:00:00,다이텍연구원,1,P-DY09,고압 래피드 염색기,RAW-001,...,69.72,26.58,80.48,71.93,31.29,80.62,3.5,5.204,NaN,2021-03-18_5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6016,6019,6017,1006,1차년도_부산대,2022-11-07 00:00:00,부산대학교,1,P-DY09,고압 래피드 염색기,RAW-029,...,49.08,56.85,51.6,51.456667,52.033333,51.963333,2.5,5.38,NaN,2022-11-07_1006
6018,6021,6019,1007,1차년도_부산대,2022-11-07 00:00:00,부산대학교,1,P-DY09,고압 래피드 염색기,RAW-029,...,49.26,56.99,51.44,51.453333,51.463333,52.39,2,6.02,NaN,2022-11-07_1007
6020,6023,6021,1008,1차년도_부산대,2022-11-07 00:00:00,부산대학교,1,P-DY09,고압 래피드 염색기,RAW-029,...,49.55,56.73,51.2,51.736667,50.963333,52.086667,2,6.23,NaN,2022-11-07_1008
6022,6025,6023,1009,1차년도_부산대,2022-11-07 00:00:00,부산대학교,1,P-DY09,고압 래피드 염색기,RAW-029,...,49.06,56.81,50.99,51.116667,53.26,52.276667,2.5,4.3,NaN,2022-11-07_1009


In [42]:
data

,index,번호,시료번호,내광성시험분석,접수번호No.,업체명,염색법,기계분류ID,기계분류,원료ID,...,L*,a*,b*,L*,a*,b*,내광성등급,색차,비고,keys
0,3,1,1,NaN,2021-03-18 00:00:00,다이텍연구원,1,P-DY09,고압 래피드 염색기,RAW-001,...,79.15,7.49,63.03,82.19,12.12,62.25,3.5,5.673333,NaN,0
1,4,2,2,NaN,2021-03-18 00:00:00,다이텍연구원,1,P-DY09,고압 래피드 염색기,RAW-001,...,75.83,14.91,74.03,78.3,18.93,72.91,3.5,4.85,NaN,0
2,5,3,3,NaN,2021-03-18 00:00:00,다이텍연구원,1,P-DY09,고압 래피드 염색기,RAW-001,...,73.7,18.95,77.57,76.54,22.88,76.71,3.5,4.93,NaN,0
3,6,4,4,NaN,2021-03-18 00:00:00,다이텍연구원,1,P-DY09,고압 래피드 염색기,RAW-001,...,72.58,21.31,78.93,74.55,26.46,79.48,3.5,5.541,NaN,0
4,7,5,5,NaN,2021-03-18 00:00:00,다이텍연구원,1,P-DY09,고압 래피드 염색기,RAW-001,...,69.72,26.58,80.48,71.93,31.29,80.62,3.5,5.204,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6021,6024,6022,1008,1차년도_부산대,2022-11-07 00:00:00,부산대학교,1,P-DY09,고압 래피드 염색기,RAW-029,...,49.55,56.73,51.2,51.736667,50.963333,52.086667,2,6.23,NaN,0
6022,6025,6023,1009,1차년도_부산대,2022-11-07 00:00:00,부산대학교,1,P-DY09,고압 래피드 염색기,RAW-029,...,49.06,56.81,50.99,51.116667,53.26,52.276667,2.5,4.3,NaN,0
6023,6026,6024,1009,1차년도_부산대,2022-11-07 00:00:00,부산대학교,1,P-DY09,고압 래피드 염색기,RAW-029,...,49.06,56.81,50.99,51.116667,53.26,52.276667,2.5,4.3,NaN,0
6024,6027,6025,1010,1차년도_부산대,2022-11-07 00:00:00,부산대학교,1,P-DY09,고압 래피드 염색기,RAW-029,...,49.65,56.96,51.58,50.916667,52.513333,51.723333,2.5,4.63,NaN,0
